In [ ]:
import os
import re
import sys
import json
import numpy as np

from scipy import interpolate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D as ax3

In [ ]:
sys.path.insert(0, '..')
from dragonfly_automation import operations, utils
from dragonfly_automation.gateway import gateway_utils

In [ ]:
gate, mms, mmc = gateway_utils.get_gate(env='prod', wrap=False)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# The list of positions generated by the HCS Site Generator plugin
position_list_filepath = \
    '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/ML0196_20191009-2//HCS_sites_20191009.pos'

In [ ]:
with open(position_list_filepath, 'r') as file:
    position_list = json.load(file)

### Old interpolation method

Here, we just measure the positions of the FocusDrive at the corners of the ROI.

In [ ]:
# parameters required for z-position interpolation
num_rows = 6
num_columns = 8
num_positions_per_well = 25

corner_z_positions = (
    (7500,  7400),  # B2, B9
    (7450,  7350),  # G2, G9
)

In [ ]:
# interpolate the positions
new_position_list_filepath, new_position_list = utils.interpolate_stage_positions_from_corners(
    position_list_filepath,
    (num_rows, num_columns),
    num_positions_per_well,
    corner_z_positions)

new_position_list_filepath

### New interpolation method

Here, we measure the focusdrive position at any number of wells and use all of them to interpolate. 

In [ ]:
# define the ROI by specifying the top left and bottom right wells 
# (for half-plate imaging, these should be B2 and G9)
top_left_well_id = 'B2'
bottom_right_well_id = 'G9'

# wells at which we usually measure positions,
# ** in the order in which they should be visited **
well_ids = [
    'B9', 'B5', 'B2',
    'E2', 
    'G2', 'G5', 'G9',
    'E9', 'E5', 
    'B9',
]

In [ ]:
# list of well_ids to consume (via list.pop) and visit
well_ids_to_visit = well_ids[::-1]

# dict, keyed by well_id, of measured FocusDrive positions
measured_focusdrive_positions = {}

In [ ]:
# find the index of the first position in a given well
def well_id_to_position_ind(well_id):
    for ind, p in enumerate(position_list['POSITIONS']):
        if p['LABEL'].startswith(well_id):
            break
    return ind

In [ ]:
# go to the next position in the well_id list
well_id = well_ids_to_visit.pop()
print('Going to well %s' % well_id)
ind = well_id_to_position_ind(well_id)
operations.go_to_position(mms, mmc, ind)

In [ ]:
# call AFC (if it is in-range) and insert the updated FocusDrive position
# in the list of measured focusdrive positions
pos_before = mmc.getPosition('FocusDrive')
mmc.fullFocus()
pos_after = mmc.getPosition('FocusDrive')

measured_focusdrive_positions[well_id] = pos_after
print('FocusDrive position before AFC: %s' % pos_before)
print('FocusDrive position after AFC: %s' % pos_after)

In [ ]:
# current xy stage position
current_pos = mmc.getPosition('XYStage')
current_pos

In [ ]:
# find the well closest the current position
dists = []
for ind, p in enumerate(position_list['POSITIONS']):
    xystage = [d for d in p['DEVICES'] if d['DEVICE'] == 'XYStage'][0]
    dist = np.sqrt(((np.array(current_pos) - np.array([xystage['X'], xystage['Y']]))**2).sum())
    dists.append(dist)
    
ind = np.argmin(dists)
well_id, site_num = utils.parse_hcs_site_label(position_list['POSITIONS'][ind]['LABEL'])
print('Nearest position is in well %s (ind = %d and distance = %d)' % (well_id, ind, min(dists)))

### Preview the interpolation

In [ ]:
measured_positions = np.array([
    (*utils.well_id_to_position(well_id), zpos) 
        for well_id, zpos in measured_focusdrive_positions.items()])

In [ ]:
interpolator = interpolate.interp2d(
    measured_positions[:, 0], 
    measured_positions[:, 1], 
    measured_positions[:, 2], 
    kind='linear')

In [ ]:
topl_x, topl_y = utils.well_id_to_position(top_left_well_id)
botr_x, botr_y = utils.well_id_to_position(bottom_right_well_id)

In [ ]:
x = np.linspace(topl_x, botr_x, 50)
y = np.linspace(topl_y, botr_y, 50)
X, Y = np.meshgrid(x, y)
Z = interpolator(x, y)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(
    X, Y, Z, rstride=1, cstride=1,
    cmap='viridis', edgecolor='none')

ax.scatter3D(
    measured_positions[:, 0], measured_positions[:, 1], measured_positions[:, 2], color='red')

### Interpolate the position list

In [ ]:
new_position_list_filepath, new_position_list = utils.interpolate_stage_positions_from_all(
    position_list_filepath,
    measured_focusdrive_positions,
    top_left_well_id,
    bottom_right_well_id)

### Visualize the results

In [ ]:
def xyz_from_pos(pos):

    well_id, site_num = utils.parse_hcs_site_label(pos['LABEL'])
    x, y = utils.well_id_to_position(well_id)
    
    focusdrive = [d for d in pos['DEVICES'] if d['DEVICE']=='FocusDrive'][0]
    z = focusdrive['X']
    
    return x, y, z

In [ ]:
pos = np.array([xyz_from_pos(p) for p in new_position_list['POSITIONS']])
pos.shape

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter3D(pos[:, 0], pos[:, 1], pos[:, 2], color='gray')
ax.scatter3D(measured_positions[:, 0], measured_positions[:, 1], measured_positions[:, 2], color='red')

In [ ]:
fig = plt.figure()
ax = plt.subplot()

ax.scatter(
    pos[:, 0], 
    pos[:, 1], 
    np.abs(pos[:, 2] - 7500) + 10, 
    color='gray')

ax.scatter(
    measured_positions[:, 0], 
    measured_positions[:, 1], 
    np.abs(measured_positions[:, 2] - 7500) + 10, 
    color='red')